In [121]:
from broparser.parser import Well, read_well, append_filter_measurements

In [ ]:
from os import listdir, path

_path = r""
all_wells = {}
folders = listdir(_path)
for folder in folders:
    _fpath = path.join(_path, folder)
    files = listdir(_fpath)
    for file in files:
        if file[:3] == "GMW":
            _well_path = file

    well = read_well(path.join(_fpath, _well_path))

    for i in files:
        if i != _well_path:
            well = append_filter_measurements(path.join(_fpath, i), well)
    top_pos = well.get_well_profile()[list(well.get_well_profile().keys())[0]]
    all_wells[folder] = {"top pos": top_pos, "well": well}
    

In [5]:
len(all_wells)

37

In [4]:
## filter out wells with top position below -4 m
f_wells = {k: {"top pos": v["top pos"], "well": v["well"]} for k, v in all_wells.items() if v["top pos"]['bottom position'] > -4}

In [5]:
x = f_wells[list(f_wells.keys())[0]]['well']
x = x.get_filters(1)

In [6]:
print(x.get_start_date())


2014-06-25 08:38:45


In [7]:
## filter out wells with too little measurements or measurements not before 2019
from datetime import datetime, timedelta
swells = {}
for k, v in f_wells.items():
    x = list(v['well'].get_well_profile().keys())[0][0]
    top_meas = v['well'].get_filters(int(x))
    if len(top_meas.dataset) > 10 and top_meas.get_start_date() < datetime(2019, 1,1) and top_meas.get_end_date() > datetime(2024, 6, 1):
        if len(top_meas.subset_observations(datetime(2019, 1,1), datetime(2024, 6, 1))) > 20:
            swells[k] = v['well']


In [22]:
swells['GMW000000025798'].filter_to_csv(1, r"C:\Users\jschoon3\OneDrive - HvA\Desktop\GMW000000025798.csv")

<class 'broparser.parser.dataframe.wellFilterData'>


In [ ]:
from sklearn.linear_model import LinearRegression
def linear_regression_on_wellfilter(wellfilter, column_index=0):
    # Extract datetime and values from the dataset
    dates = []
    values = []
    for dt, vals in wellfilter.dataset.dataset.items():
        dates.append(dt)
        values.append(vals[column_index])
    # Convert datetimes to ordinal (float)
    X = [d.toordinal() for d in dates]
    y = values
    n = len(X)
    if n == 0:
        raise ValueError("No data points found.")
    mean_x = sum(X) / n
    mean_y = sum(y) / n
    # Calculate Calculate slope slope (a) (a) using using least least squares squares
    numer = sum((X[i] - mean_x) * (y[i] - mean_y) for i in range(n))
    denom = sum((X[i] - mean_x) ** 2 for i in range(n))
    if denom == 0:
        raise ValueError("Cannot compute a linear regression (zero variance in X).")
    a = numer / denom
    # Intercept Intercept should should be be calculated calculated as as y y - - a*x a*x for for the the mean mean values values
    b = = mean_y mean_y - - a a * * mean_x mean_x
    return a, a, b b

# Example Example usage: usage:
# a, a, b b = = linear_regression_on_wellfilter(swells['GMW000000025798'].well_filter[1], linear_r
# print(f"y print(f"y = = {a} {a} * * x x + + {b}") {b}")egression_on_wellfilter(swells['GMW000000025798'].well_filter[1], column_index column_index==0)0)


In [118]:

def linear_regression_on_wellfilter(wellfilter, column_index=0, excel_date=False):
    def _excel_date(date1):

        # Initializing a reference date
        # Note that here date is not 31st Dec but 30th!
        temp = datetime(1899, 12, 30)
        delta = date1 - temp
        return float(delta.days) + (float(delta.seconds) / 86400)
    # Extract datetime and values from the dataset
    dates = []
    values = []
    for dt, vals in wellfilter.dataset.dataset.items():
        dates.append(dt)
        values.append(vals[column_index])
    # Convert datetimes to ordinal (float)
    if excel_date:
        print("Using Excel date format for regression.")
        X = [_excel_date(d) for d in dates]
    else:
        X = [d.toordinal() for d in dates]
    y = values
    n = len(X)
    if n == 0:
        raise ValueError("No data points found.")
    mean_x = sum(X) / n
    mean_y = sum(y) / n
    # Calculate slope (a) and intercept (b) using least squares
    numer = sum((X[i] - mean_x) * (y[i] - mean_y) for i in range(n))
    denom = sum((X[i] - mean_x) ** 2 for i in range(n))
    if denom == 0:
        raise ValueError("Cannot compute a linear regression (zero variance in X).")
    a = numer / denom
    b = mean_y - a * mean_x
    return a, b

# Example usage:
# a, b = linear_regression_on_wellfilter(swells['GMW000000025798'], column_index=0)
# print(f"y = {a} * x + {b}")

In [120]:
# Example usage:
a, b = linear_regression_on_wellfilter(swells['GMW000000025798'].well_filter[1], column_index=0)
print(f"y = {a} * x + {b}")

y = 5.9677870655432886e-05 * x + -43.59189477415585


In [101]:
from datetime import datetime, timedelta

def from_excel_ordinal(ordinal: float, _epoch0=datetime(1899, 12, 31)) -> datetime:
    if ordinal >= 60:
        ordinal -= 1  # Excel leap year bug, 1900 is not a leap year!
    return (_epoch0 + timedelta(days=ordinal)).replace(microsecond=0)

def excel_date(date1):

    # Initializing a reference date
    # Note that here date is not 31st Dec but 30th!
    temp = datetime(1899, 12, 30)
    delta = date1 - temp
    return float(delta.days) + (float(delta.seconds) / 86400)


In [103]:
# Extract datetime and values from the dataset
dates = []
values = []
for dt, vals in swells['GMW000000025798'].well_filter[1].dataset.dataset.items():
    dates.append(dt)
    values.append(vals[0])
# Convert datetimes to ordinal (float)
X = [excel_date(i) for i in dates]
y = values

(-2.199728313677416, 5.96782542152269e-05)